# 09. 최종 통합 위험도 평가

## 목적
- 클러스터링 + 시간라벨 + 배달분석 통합
- 최종 위험도 스코어 계산 (0-100점)
- 타겟별 맞춤 정책 제안

In [1]:
import pandas as pd
import numpy as np

labeled_data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/1_시간라벨링/labeled_data.csv"
df = pd.read_csv(labeled_data_path)

cluster_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석/클러스터링_결과_완전판.csv"
cluster_df = pd.read_csv(cluster_path)

df = df.merge(cluster_df[['가맹점구분번호', 'cluster']], on='가맹점구분번호', how='left')
df['위험그룹'] = df['cluster'].apply(lambda x: '고위험' if x in [0, 2] else '정상' if x == 1 else '알수없음')

print(f"통합 데이터: {len(df):,}개")

통합 데이터: 89,412개


## 최종 위험도 스코어 계산 (0-100점)

In [2]:
# 클러스터 기반 위험도 (0-40점)
def cluster_risk_score(cluster):
    if cluster == 0:
        return 40
    elif cluster == 2:
        return 35
    elif cluster == 1:
        return 10
    else:
        return 0

df['클러스터위험점수'] = df['cluster'].apply(cluster_risk_score)

# 시간 라벨 기반 긴급도 (0-30점)
def time_urgency_score(status):
    if status == 'D-3m':
        return 30
    elif status == 'D-6m':
        return 25
    elif status == 'D-9m':
        return 20
    elif status == 'D-12m':
        return 15
    elif status == '정상영업':
        return 5
    else:
        return 0

if '상태' in df.columns:
    df['긴급도점수'] = df['상태'].apply(time_urgency_score)
else:
    df['긴급도점수'] = 0

# 최종 위험도 (0-100점)
df['최종위험도'] = df['클러스터위험점수'] + df['긴급도점수']

print("\n[최종 위험도 분포]")
print(df['최종위험도'].describe())


[최종 위험도 분포]
count    89412.000000
mean        44.897609
std          2.317931
min         15.000000
25%         45.000000
50%         45.000000
75%         45.000000
max         70.000000
Name: 최종위험도, dtype: float64


## 위험 등급 분류

In [3]:
def classify_final_risk(score):
    if score >= 80:
        return '최고위험'
    elif score >= 60:
        return '고위험'
    elif score >= 40:
        return '중위험'
    elif score >= 20:
        return '저위험'
    else:
        return '정상'

df['최종위험등급'] = df['최종위험도'].apply(classify_final_risk)

print("\n[최종 위험 등급 분포]")
print(df['최종위험등급'].value_counts().sort_index())


[최종 위험 등급 분포]
최종위험등급
고위험      365
저위험      447
정상       143
중위험    88457
Name: count, dtype: int64


### 종합 결론

## 최종 위험도 평가 체계
- **최고위험 (80+)**: 즉시 개입 필요
- **고위험 (60-79)**: 주간 모니터링
- **중위험 (40-59)**: 월간 점검
- **저위험 (20-39)**: 분기별 관찰
- **정상 (<20)**: 연간 평가

## 정책 활용
1. 위험도 기반 자원 배분
2. 타겟별 맞춤 정책 수립
3. 조기 경보 시스템 운영